# Minería de Textos
# Cuaderno FLAN

En este cuaderno se mostrará como realizar un instruction tunning a [FLAN-T5](https://huggingface.co/docs/transformers/model_doc/flan-t5).


# Instalación e importación de librerías y definición de parámetros

Se instalan a continuación las librerías necesarias para la ejecución de este cuaderno.

In [ ]:
%%capture

!pip install transformers==4.27.2
!pip install datasets==2.15.0
!pip install tqdm==4.66.1
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install wandb
!pip install accelerate

Se importan las librerías necesarias para este cuaderno.

In [ ]:
from transformers import T5ForConditionalGeneration, AutoTokenizer, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, Dataset
from tqdm import tqdm

Una vez importadas, se definen una serie de parámetros que se utilizarán a lo largo del cuaderno.

- **model_name**: Nombre del modelo que será reentrenado.
- **dataset_name**: Nombre del dataset utilizado.
- **epochs**: Número de épocas de entrenamiento.
- **dropout**: Porcentaje de dropout.
- **max_length**: Longitud máxima de las secuencias de entrada. El máximo del modelo son 512.
- **use_cuda**: Booleano que indica si se utilizará la GPU para entrenar.
- **results_dir**: Directorio donde se guardarán los resultados.

In [ ]:
args = {}
args['model_name'] = "google/flan-t5-small"
args['dataset_name'] = "ncbi_disease"
args['epochs'] = 5
args['dropout'] = 0.2
args['max_length'] = 200
args['use_cuda'] = True
args['results_dir'] = "experiments/test"

# Carga y procesado del dataset

Mediante la función `load_dataset` de la librería `datasets`, se puede cargar un dataset de manera sencilla que esté publicado en HuggingFace.

En este caso se va cargar el dataset `ncbi_disease` que contiene 7934 abstracts de PubMed con anotaciones de enfermedades. Toda la información se encuentra en [este enlace](https://huggingface.co/datasets/ncbi_disease).

In [ ]:
dataset = load_dataset(args['dataset_name'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/5433 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/924 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/941 [00:00<?, ? examples/s]

Este dataset contiene los datos divididos en 3 particiones: `train`, `validation` y `test`. Se utilizará la partición de `train` para entrenar el modelo, la de `validation` para validar el modelo y la de `test` para evaluar el modelo.

Las columnas que contiene el dataset son las siguientes:

- **id**: Identificador de la frase.
- **tokens**: Lista de tokens que componen la frase.
- **ner_tags**: Lista de etiquetas, donde el 0 indica que ese token no es una enfermedad, el 1 indica el inicio de una enfermedad, y el 3 indica que ese token es la continuación de la enfermedad.

Se define la función `transformElements` que se encargará de transformar los datos del dataset para que puedan ser utilizados por el modelo. Esta función junta los tokens de la frase en un solo string, y el texto correspondiente a la enfermedad en otro string.

In [ ]:
def transformElement(element):
    text = ' '.join(element['tokens'])
    annText = ''

    for i, elem in enumerate(element['ner_tags']):
        if elem != 0:
            annText += element['tokens'][i] + ' '

    annText = annText.strip()

    return {'text' : text, 'annText' : annText}


Se aplica la función definida a cada elemento del dataset mediante la función `map` de la librería `datasets`.

In [ ]:
datasetTransformed = dataset.map(transformElement, remove_columns=["id", "tokens", "ner_tags"])

Map:   0%|          | 0/5433 [00:00<?, ? examples/s]

Map:   0%|          | 0/924 [00:00<?, ? examples/s]

Map:   0%|          | 0/941 [00:00<?, ? examples/s]

In [ ]:
datasetTransformed['train'][0]

{'text': 'Identification of APC2 , a homologue of the adenomatous polyposis coli tumour suppressor .',
 'annText': 'adenomatous polyposis coli tumour'}

# Reentrenamiento del modelo

Se define la función `preprocess_function` que se encargará de procesar los datos de entrada para que puedan ser utilizados por el modelo.

Esta función, usando el tokenizador, genera el prompt a pasarle al modelo y lo codifica, añadiendo a la entrada también la enfermedad a predecir.

<span style="color:red">Atención:</span> Esta función está definida para poder ser usada de forma paralela (`batched=True`) al ser usada por la función `map` de la librería `datasets`. Esto permite que el procesado de los datos sea más rápido, al procesar varios elementos a la vez. Es por ello que te pueda parecer que la función está definida de forma extraña.

In [ ]:
def preprocess_function(sample, padding="max_length"):

    # Generar los prompts para el modelo dado el texto de cada elemento del dataset
    inputs = [
        'Given the sentence : "' +
        item.replace("\n", " ") +
        '", the annotated disease text is: "'
        for item in sample['text']
    ]

    # Usar el tokenizador para generar los inputs del modelo para cada elemento del dataset, dado el texto
    model_inputs = tokenizer(
        inputs,
        max_length=args['max_length'],
        padding=padding,
        truncation=True,
    )

    # Usar el tokenizador para generar los targets del modelo para cada elemento del dataset, dado el texto anotado
    target_diseases = tokenizer(
        text_target=sample["annText"],
        max_length=args['max_length'],
        padding=padding,
        truncation=True,
    )

    # Si se está haciendo padding (es decir, si se está fijando el tamaño máximo de la secuencia de entrada y no considerando el
    # tamaño de cada secuencia de entrada), se reemplazan todos los tokens de padding por -100 para que no se consideren en la
    # función de pérdida. Esto se realiza para que el modelo ignore los tokens de padding en la función de pérdida y no se
    # penalice por ellos, ni se aprenda a predecirlos.
    if padding == "max_length":
        target_diseases["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label]
            for label in target_diseases["input_ids"]
        ]

    # Se agregan los targets al diccionario de inputs del modelo
    model_inputs["labels"] = target_diseases["input_ids"]

    # Se retorna el diccionario de inputs del modelo
    return model_inputs

Se importa el tokenizador y el modelo.

El tokenizador se importa haciendo uso de la función `AutoTokenizer` de la librería `transformers`. Esta función se encarga de cargar el tokenizador adecuado para el modelo que se le pasa como parámetro.

El modelo se carga haciendo uso de la función `T5ForConditionalGeneration` de la librería `transformers`. Esta función se encarga de cargar el modelo T5 para fine-tuning, especificando que la tarea a realizar es la de generación condicional. (Más información en [este enlace](https://huggingface.co/docs/transformers/model_doc/t5#transformers.T5ForConditionalGeneration))

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(args['model_name'])
model = T5ForConditionalGeneration.from_pretrained(args['model_name'])

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Una vez se tiene el tokenizador y la función `preprocess_function` definidos, se aplica esa función sobre todo el dataset mediante la función `map` de la librería `datasets`.

In [ ]:
tokenized_dataset = datasetTransformed.map(
    preprocess_function, batched=True, remove_columns=["text", "annText"]
)

Map:   0%|          | 0/5433 [00:00<?, ? examples/s]

Map:   0%|          | 0/924 [00:00<?, ? examples/s]

Map:   0%|          | 0/941 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset['train'][0]

{'input_ids': [9246,
  8,
  7142,
  3,
  10,
  96,
  21153,
  2420,
  13,
  71,
  4051,
  357,
  3,
  6,
  3,
  9,
  13503,
  10384,
  13,
  8,
  3,
  9,
  537,
  23926,
  302,
  4251,
  19882,
  7,
  3,
  9044,
  29851,
  18513,
  127,
  3,
  535,
  6,
  8,
  46,
  2264,
  920,
  1994,
  1499,
  19,
  10,
  96,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

Se indica si se va a utilizar la GPU para entrenar el modelo.

In [ ]:
if args['use_cuda']:
    model.cuda()

Se definen los hiperparámetros de entrenamiento utilizando la clase `Seq2SeqTrainingArguments` de la librería `transformers`. Esta clase permite definir los hiperparámetros de entrenamiento de una manera sencilla. (Más información en [este enlace](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments)).

En este caso se han definido los siguientes hiperparámetros:

- **do_train**: Booleano que indica si se va a entrenar el modelo. En este caso se va a entrenar el modelo.
- **do_eval**: Booleano que indica si se va a evaluar el modelo usando el dataset de validación. En este caso se va a evaluar el modelo.
- **evaluation_strategy**: Estrategia de evaluación. En este caso se evaluará al final de cada época.
- **logging_strategy**: Estrategia de logging (mostrar mensajes). En este caso se hará logging al final de cada época.
- **save_strategy**: Estrategia de guardado. En este caso no se guardará el modelo al final de cada época, sino que se guardará el mejor modelo.
- **per_device_train_batch_size**: Tamaño del batch de entrenamiento por dispositivo.
- **per_device_eval_batch_size**: Tamaño del batch de evaluación por dispositivo.
- **auto_find_batch_size**: Booleano que indica si se va a buscar el tamaño de batch más grande posible para entrenar el modelo. En este caso se va a buscará.
- **gradient_accumulation_steps**: Número de pasos de acumulación de gradientes. Esto se suele utilizar al entrenar modelos grandes, los cuales requieren de mucha memoria. En este caso se acumulará un paso de gradiente.
- **learning_rate**: Tasa de aprendizaje. En este caso se utilizará una tasa de aprendizaje de 1e-5.
- **num_train_epochs**: Número de épocas de entrenamiento.
- **output_dir**: Directorio donde se guardarán los resultados.
- **logging_dir**: Directorio donde se guardarán los logs.

In [ ]:
training_args = Seq2SeqTrainingArguments(
    do_train=True,
    do_eval=True,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="no",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    auto_find_batch_size=True,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    num_train_epochs=args['epochs'],
    output_dir=args['results_dir'],
    logging_dir=f"{args['results_dir']}/logs",
)

Se define ahora el _data collator_, que es el encargado de procesar los datos de entrada para que puedan ser utilizados por el modelo. En este caso se utiliza la clase `DataCollatorForSeq2Seq` de la librería `transformers`. Esta clase se encarga de procesar los datos de entrada para que puedan ser utilizados por el modelo T5. (Más información en [este enlace](https://huggingface.co/docs/transformers/main_classes/data_collator#transformers.DataCollatorForSeq2Seq)).

Como se quieren ignorar los tokens de padding, en el parámetro `label_pad_token_id` se indica el valor -100 para que se ignoren. Ignorando los tokens de padding, se consigue que el modelo no tenga en cuenta los tokens de padding a la hora de calcular la pérdida.

Respecto al valor de `pad_to_multiple_of`, se indica el valor 8 para que el tamaño de las secuencias de entrada sea múltiplo de 8. Esto es necesario para poder utilizar la GPU para entrenar el modelo.

In [ ]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=-100,
    pad_to_multiple_of=8,
)

Ahora, se define el objeto para reentrenar el modelo, en este caso se utiliza la clase `Seq2SeqTrainer` de la librería `transformers`. Esta clase se encarga de reentrenar un modelo T5. (Más información en [este enlace](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainer)).

En él se define el modelo a usar, los argumentos de entrenamiento, el _data collator_ y el dataset de entrenamiento y de validación.

Además, mediante la orden `model.config.use_cache = False` se indica que no se utilice la caché del modelo. Haciendo esto, se consigue que el modelo no utilice la caché de las capas de atención que ya conocía, por tanto, los parámetros del modelo se actualizarán en base a los nuevos datos.

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator
)

model.config.use_cache = False


Se ejecuta el reentrenamiento del modelo.

La pérdida se calcula utilizando la función de entropía cruzada [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html).

Atención: Puede ser que te pida una API Key de WandB. WandB (Weights & Biases) es una plataforma integral para el seguimiento y visualización de experimentos de aprendizaje automático. Facilita el registro y comparación de hiperparámetros y métricas, así como la colaboración entre equipos al proporcionar un espacio centralizado para compartir resultados y códigos. Con integración fácil en bibliotecas populares, WandB se destaca por su capacidad para mejorar la eficiencia en la gestión y comprensión de modelos, convirtiéndose en una herramienta valiosa para profesionales de aprendizaje automático.

In [ ]:
modelTrainer = trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,0.927400,0.436500
2,0.571800,0.361529
3,0.497000,0.329960
4,0.462900,0.304799
5,0.434000,0.304884


Una vez reentrenado, se guarda el modelo y el tokenizador en el directorio `results_dir`.

In [ ]:
trainer.model.save_pretrained(args['results_dir'])
tokenizer.save_pretrained(args['results_dir'])

('experiments/test/tokenizer_config.json',
 'experiments/test/special_tokens_map.json',
 'experiments/test/tokenizer.json')

# Inferencia

Para realizar la inferencia, se utiliza el modelo ya reentrenado y el tokenizador. Se importan a continuación.

In [ ]:
model = T5ForConditionalGeneration.from_pretrained('/content/experiments/test')
tokenizer = AutoTokenizer.from_pretrained('/content/experiments/test')

Ahora, se crea un diccionario que contendrá todos los prompts, junto con sus correspondientes enfermedades, que se utilizarán para realizar la inferencia.

In [ ]:
datasetPrompts = {'train':[], 'validation':[], 'test':[]}
for split in datasetTransformed.keys():
    for element in datasetTransformed[split]:
        auxElement = {'text': '', 'annText': element['annText']}
        auxElement['text'] = 'Given the sentence : "' + element['text'].replace("\n", " ") + '", the annotated disease text is: "'
        datasetPrompts[split].append(auxElement)

Se crea una función auxiliar llamada `generateOutput`, en la que dado el modelo, el tokenizador y un prompt, se genera la salida del modelo.

In [ ]:
def generateOutput(model, tokenizer, prompt):

    #Se tokeniza el prompt
    inputs = tokenizer(prompt, return_tensors="pt")

    #Se obtiene la salida del modelo, dados:
    # - los inputs tokenizados
    # - la máscara de atención
    # - la longitud máxima de la secuencia de salida, establecida en este caso en 32
    # - el número de beams a usar en la decodificación, establecido en este caso en 5. Un beam es una hipótesis de salida
    #   que el modelo considera como una posible solución al problema. El modelo genera varias hipótesis de salida y
    #   selecciona la mejor de ellas como la salida final.
    # - early_stopping=True para que el modelo deje de generar hipótesis de salida cuando todas las hipótesis generadas
    #   tengan el token de fin de secuencia (</s>) o cuando se haya generado el número máximo de hipótesis de salida
    outputs = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_length=32,
        num_beams=5,
        early_stopping=True,
            return_dict_in_generate=True

    )

    decoded_output = tokenizer.decode(outputs[0][0], skip_special_tokens=True)

    return decoded_output

Se puede comprobar con el siguiente código que la inferencia funciona correctamente en un único ejemplo.

In [ ]:
i = 3
generated = generateOutput(model, tokenizer, datasetPrompts['train'][i]['text'])
real = datasetPrompts['train'][i]['annText']

print(f"Generated: {generated}")
print(f"Real: {real}")
print(f"Is the generated text the same as the real one? Yes" if generated == real else "Is the generated text the same as the real one? No")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


Generated: colon carcinoma
Real: colon carcinoma
Is the generated text the same as the real one? Yes


Se define una función llamada `getMatchMetric` para que realice una métrica de coincidencia entre la salida del modelo y la enfermedad real.

Esta función mide las coincidencias exactas que otorga el modelo, así como las parciales (que lo inferido se encuentre dentro de la enfermedad real o viceversa).

Se retornan las coincidencias exactas, parciales, y el número de coincidencias exactas con respecto al total.

In [ ]:
def getMatchMetric(model, tokenizer, split):
    matches = 0
    partialMatch = 0

    for element in tqdm(split):
        output = generateOutput(model, tokenizer, element['text'])
        if output == element['annText']:
            matches += 1
        elif output in element['annText']:
            partialMatch += 1
        elif element['annText'] in output:
            partialMatch += 1
    return {'matches': matches, 'partialMatch': partialMatch, 'total': len(split), 'accuracy': matches/len(split)}

Se ejecutan las métricas y se muestran los resultados.

In [ ]:
print("Evaluating training set...")
trainMetric = getMatchMetric(model, tokenizer, datasetPrompts['train'])
print("Train matches: " + str(trainMetric['matches']))
print("Train partial matches: " + str(trainMetric['partialMatch']))
print("Train accuracy: " + str(trainMetric['accuracy']))

print("Evaluating validation set...")
validationMetric = getMatchMetric(model, tokenizer, datasetPrompts['validation'])
print("Validation matches: " + str(validationMetric['matches']))
print("Validation partial matches: " + str(validationMetric['partialMatch']))
print("Validation accuracy: " + str(validationMetric['accuracy']))

print("Evaluating test set...")
testMetric = getMatchMetric(model, tokenizer, datasetPrompts['test'])
print("Test matches: " + str(testMetric['matches']))
print("Test partial matches: " + str(testMetric['partialMatch']))
print("Test accuracy: " + str(testMetric['accuracy']))

Evaluating training set...


100%|██████████| 5433/5433 [1:21:27<00:00,  1.11it/s]


Train matches: 3478
Train partial matches: 1448
Train accuracy: 0.6401619731271857
Evaluating validation set...


100%|██████████| 924/924 [12:44<00:00,  1.21it/s]


Validation matches: 577
Validation partial matches: 255
Validation accuracy: 0.6244588744588745
Evaluating test set...


100%|██████████| 941/941 [14:48<00:00,  1.06it/s]

Test matches: 565
Test partial matches: 278
Test accuracy: 0.6004250797024442
